In [1]:
#!pip install biotite
import biotite as bt
import biotite.database.rcsb as rcsb
import requests
import os
import gzip
import io
import shutil

In [2]:
# RET: P07949 (RET_HUMAN)
query1 = rcsb.FieldQuery("rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession", exact_match ='P07333')
query2 = rcsb.FieldQuery("reflns.d_resolution_high", less=2.5)
final_query = query1 & query2
RET_pdb_ids = rcsb.search(final_query)
print(RET_pdb_ids)

RequestError: Error 404: HTTP 404 Not Found

In [ ]:
os.getcwd()

In [ ]:
#passing pdb that i want to download as a list :
RET_pdb_ids =['2wqb']

In [ ]:
#downloading the structures from pdbrenum in .gz format
for id in RET_pdb_ids:
    download_url_prefix = 'http://dunbrack3.fccc.edu/PDBrenum/output_PDB/'
    download_url_suffix = '_renum.pdb.gz'
    pdb_id = id.lower()
    print(pdb_id)
    download_url = download_url_prefix + pdb_id + download_url_suffix
    print(download_url)
    response = requests.get(download_url)
    print(response.status_code)
    fname = pdb_id + '.pdb.gz'
    with open(fname, 'wb') as f:
        f.write(response.content)
    print(f'completed writing {fname}..')

In [ ]:
all = os.listdir()
for f in all:
    if f.endswith('.pdb.gz'):
        try:
            with gzip.open(f, 'rb') as ip:
                with io.TextIOWrapper(ip, encoding='utf-8') as decoder:
                    # Let's read the content using read()
                    content = decoder.read()
                    #base = f[:-4]
                    fname = f[:-3]
                    #print(content)
                    with open(fname,'w') as file_obj:
                        file_obj.write(content)
                print(f'converted {f}===>>{fname}')
        except:
            print(f'this file {f} is not a zip file ')

In [ ]:
#this file 7du9.pdb.gz is not a zip file 
#this file 7dua.pdb.gz is not a zip file
#
#
#

In [ ]:
######2. P11362 (FGFR1_HUMAN)
######3. P21802 (FGFR2_HUMAN)
#####4.P22607 (FGFR3_HUMAN)
#####5.P22455 (FGFR4_HUMAN)
###6.P17948 (VGFR1_HUMAN)
####7 P36888 (FLT3_HUMAN)
###8 P35916 (VGFR3_HUMAN)
###9 P35968 (VGFR2_HUMAN)
##10 P07333 (CSF1R_HUMAN)
#11  P10721 (KIT_HUMAN)
##12 P16234 (PGFRA_HUMAN)
##13 P09619 (PGFRB_HUMAN)
##14  P35590 (TIE1_HUMAN)
###15 Q02763 (TIE2_HUMAN)

In [ ]:
def write_list_to_file(list_name,file_name):
    file = open(file_name,'x')
    with open(file_name,'a') as f:
        for i in list_name:
            line = i + '\n'
            f.write(line)
    return()

In [ ]:
write_list_to_file(RET_pdb_ids,'test.txt')

In [ ]:
def get_structures(base_name ,uniprot_id):
    ## stage 1 make an empt list for .gz files that dont open
    error_list =[]
    ## stage 2: making queries using the uniprot id and then filter for structures with good resolution using query 2
    query1 = rcsb.FieldQuery("rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
                             exact_match = uniprot_id)
    query2 = rcsb.FieldQuery("reflns.d_resolution_high", less=2.5)
    final_query = query1 & query2 # combining queries
    list_of_pdb_ids = base_name + 'pdb_ids' # this will be the list that will hold the results
    list_of_pdb_ids = rcsb.search(final_query)
    print(f'generated a list of pdb_Ids by searching using {uniprot_id} on the rcsb database')
    ### next step is to build a http address for each result and download them from pdb renum
    for id in list_of_pdb_ids:
        print(f'preparing to download {id} structure as .gz file')
        download_url_prefix = 'http://dunbrack3.fccc.edu/PDBrenum/output_PDB/'
        download_url_suffix = '_renum.pdb.gz'
        pdb_id = id.lower() # the pdb ids need to be in lower cases as it is used like that in pdbrenum
        #print(pdb_id)
        download_url = download_url_prefix + pdb_id + download_url_suffix
        #print(download_url)
        response = requests.get(download_url)
        print(response.status_code) # 200 means all clear
        fname = pdb_id + '.pdb.gz' # the downloaded file will be .gz not pdb !
        ######### below is the loop for writing the .gz files
        with open(fname, 'wb') as f:
            f.write(response.content)
        print(f'completed writing {fname}..')
    ##########################
    all = os.listdir()
    ###### below is the loop for uncompressing the .gz files..
    for f in all:
        print(f'looping through {f}..')
        if f.endswith('.pdb.gz'):
            print(f'working to unzip {f}....')
            try:
                with gzip.open(f, 'rb') as ip:
                    with io.TextIOWrapper(ip, encoding='utf-8') as decoder:
                        # Let's read the content using read()
                        content = decoder.read()
                        #base = f[:-4]
                        fname = f[:-3]
                        #print(content)
                        with open(fname,'w') as file_obj:
                            file_obj.write(content)
                            print(f'converted {f}===>>{fname}')
            except:
                print(f'this file {f} is not a zip file ') # note some .gz files from the site dont uncompress
                error_list.append(f) # those file names will be captured here
    ##################### next steps creating a folder with base name in cwd 
    ############# moving .pdb files to that folder, deleting the .gz files
    home = os.getcwd()
    all = os.listdir()
    os.mkdir(base_name)
    for f in all:
        if f.endswith('.pdb'):
            dest_folder = os.path.join(home,base_name,f)
            source_folder = os.path.join(home,f)
            shutil.move(source_folder,dest_folder)
        elif f.endswith('pdb.gz'):
            os.remove(f) 
    ######### below code is using the other function (write list to file) to write search results to file
    output_fname = base_name + '.txt'
    print(f'writing search results to file named {output_fname}')
    write_list_to_file(list_of_pdb_ids,output_fname)
    ### below code writer .err file
    error_fname = base_name + '.err'
    print(f'writing error ids to file named {error_fname}')
    write_list_to_file(error_list,error_fname)
    print(f'job for {base_name} with uniprot id : {uniprot_id} is complete')
    return()

In [ ]:
home = os.getcwd()
all = os.listdir()
folder = 'RET_HUMAN'
#os.mkdir(folder)
for f in all:
    if f.endswith('.pdb'):
        dest_folder = os.path.join(home,folder,f)
        source_folder = os.path.join(home,f)
        shutil.move(source_folder,dest_folder)
    elif f.endswith('pdb.gz'):
        os.remove(f)
        
        

In [ ]:
######2. P11362 (FGFR1_HUMAN)
######3. P21802 (FGFR2_HUMAN)
#####4.P22607 (FGFR3_HUMAN)
#####5.P22455 (FGFR4_HUMAN)
###6.P17948 (VGFR1_HUMAN)
####7 P36888 (FLT3_HUMAN)
###8 P35916 (VGFR3_HUMAN)
###9 P35968 (VGFR2_HUMAN)
##10 P07333 (CSF1R_HUMAN)
#11  P10721 (KIT_HUMAN)
##12 P16234 (PGFRA_HUMAN)
##13 P09619 (PGFRB_HUMAN)
##14  P35590 (TIE1_HUMAN)
###15 Q02763 (TIE2_HUMAN)

In [ ]:
# RET: P07949 (RET_HUMAN)
#P21802 (GFR2_HUMAN)
#P22607 (FGFR3_HUMAN)----very few structures
#P22455 (FGFR4_HUMAN)
#P17948 (VGFR1_HUMAN)
#7  ('FLT3_HUMAN','P36888')
###8 P35916 ('VGFR3_HUMAN','P35916')-- no highres structures # only one structure with res of 4 A
###9 P35968 ('VGFR2_HUMAN','P35968')--many structures
##10 P07333 ('CSF1R_HUMAN','P07333')--many structures
#11  P10721 ('KIT_HUMAN','P10721')----many structures
##12 P16234 ('PGFRA_HUMAN','P16234')--few structures
##13 P09619 ('PGFRB_HUMAN','P09619')--few structures
##14  P35590 ('TIE1_HUMAN','P35590')--no structures--only one structure of fibronectin domain
###15 Q02763 ('TIE2_HUMAN','Q02763')--few structures
####### P00533 · EGFR_HUMAN
####   Q9UM73 · ALK_HUMAN -- no structures that meet refinement criteria
###  Q08345 · DDR1_HUMAN

In [ ]:
for index, tuple in enumerate(list_of_tuples):
	element_one = tuple[0]
	element_two = tuple[1]
	print(element_one, element_two)

In [ ]:
vgfr1

In [ ]:
get_structures('TIE2_HUMAN','Q02763')

In [ ]:
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
os.chdir('pdb-structures')

In [ ]:
os.walk("./")

In [ ]:
for cur_dir, sub_dirs,files in os.walk("./"):
    print(f"#########processing {cur_dir}##############")
    print("sub-dirs",sub_dirs)
    print("files",files)
    print(len(files))

https://www.ebi.ac.uk/pdbe/pdbe-kb/proteins/Q02763/structures # use this link for checking pdb coverage for individual uniprot ids


In [9]:
import os
#from chimerax.core.commands import run as rc
for i in range (1,13):
    print(i)
    modelname = '#' + str(i)
    fname = str(i) + '.pdb'
    print(modelname)
    #rc(session,"save " + fname + modelname )
    print(f'"save " + {fname} + {modelname}')

1
#1
"save " + 1.pdb + #1
2
#2
"save " + 2.pdb + #2
3
#3
"save " + 3.pdb + #3
4
#4
"save " + 4.pdb + #4
5
#5
"save " + 5.pdb + #5
6
#6
"save " + 6.pdb + #6
7
#7
"save " + 7.pdb + #7
8
#8
"save " + 8.pdb + #8
9
#9
"save " + 9.pdb + #9
10
#10
"save " + 10.pdb + #10
11
#11
"save " + 11.pdb + #11
12
#12
"save " + 12.pdb + #12


In [3]:
print('hello')

hello
